# Хеш-функции


## Введение

**Определение:** Хеш-функция $h$ - отображение из произвольного множества входных данных $X$ в группу остатков от деления на $n$: $\mathbb{Z}_n = \{0, ..., n-1\}$

$$h: X \rightarrow \mathbb{Z}_n, \quad n \le |X|$$


**Применения хеш-функций**

- Хеш-таблицы - эффективная структура словаря ключ-значение (самая часто используемая структура данных, за исключением массива)

- В протоколах и криптографии - например хеширование паролей при сохранении в БД или чек-суммы в протоколах передачи данных

- Сравнение объектов - например проверка графов на изоморфизм или быстрое сравнение подстрок

- Источник детерминированного рандома - например если мы хотим распределять сообщения к 5 инстансами микросервиса, чтобы при этом сообщения от одного и того же пользователя попадали на один и тот же инстанс, мы можем брать хеш от его ip

- Поиск в многомерных пространствах - детерминированный поиск ближайшей точки среди $m$ точек в $n$-мерном пространстве быстро не решается. Однако можно придумать хеш-функцию, присваивающую лежащим рядом элементам одинаковые хеши, и делать поиск только среди элементов с тем же хешом, что у запроса (local-sensitive hashing)

**Свойства хэш-функций**

- Детерминированность - одни и те же входные данные всегда должны давать одинаковый результат, т.е. $h$ действительно функция))

- Вычислимость - как минимум, $h$ должна быть вычислимой функцией. Еще лучше если она вычисляется быстро, например за $O(n)$

- Равномерность - $\forall y \in Z_n$ $P(h(x) = y) \le \frac{c}{n}$, где $c$ - константа

- Лавинность - небольшое изменение входных данных должно приводить к значительному изменению хэш-значения

**Теорема:**
Пусть $h: X \longrightarrow \mathbb{Z}_m$ - равномерная хеш-функция, тогда

вероятность коллизии $$P(n, m) \approx 1 - e^{-\frac{n(n-1)}{2m}}$$


При $n \approx 1.2\sqrt{m}$ вероятность коллизии составляет примерно 0.5.

$\square$

Вероятность того, что все $n$ элементов имеют различные хэш-значения, равна:

$$P(\text{нет коллизий}) = \frac{m!}{m^n \cdot (m-n)!} = \frac{m \cdot (m-1) \cdot ... \cdot (m-n+1)}{m^n}$$

Используя приближение $(1-x) \approx e^{-x}$ для малых $x$, получаем:

$$P(\text{нет коллизий}) \approx \prod_{i=0}^{n-1} (1 - \frac{i}{m}) \approx \prod_{i=0}^{n-1} e^{-\frac{i}{m}} = e^{-\sum_{i=0}^{n-1} \frac{i}{m}} = e^{-\frac{n(n-1)}{2m}}$$

Следовательно, вероятность коллизии:

$$P(\text{коллизия}) = 1 - P(\text{нет коллизий}) \approx 1 - e^{-\frac{n(n-1)}{2m}}$$

$\blacksquare$

## Типы хэш-функций

### 1. Хэш-функции для хэш-таблиц

Эти функции оптимизированы для быстрого вычисления и равномерного распределения значений. Они не обязательно обладают криптографической стойкостью.

#### Метод деления

$$h(k) = k \mod m$$

где $k$ - ключ, $m$ - размер хэш-таблицы.

#### Метод умножения

$$h(k) = \lfloor m \cdot (k \cdot A \mod 1) \rfloor$$

где $A$ - константа, обычно выбираемая как $A \approx \frac{\sqrt{5}-1}{2} \approx 0.618$.

#### Метод универсального хэширования

$$h_{a,b}(k) = ((a \cdot k + b) \mod p) \mod m$$

где $p$ - простое число, большее максимального значения ключа, $a$ и $b$ - случайные числа, $1 \leq a < p$ и $0 \leq b < p$.


In [ ]:
# Реализация различных методов хэширования
def division_hash(key, size):
    """Метод деления"""
    return key % size

def multiplication_hash(key, size):
    """Метод умножения"""
    A = (math.sqrt(5) - 1) / 2  # Константа ≈ 0.618
    return int(size * ((key * A) % 1))

def universal_hash(key, size, a, b, p):
    """Универсальное хэширование"""
    return ((a * key + b) % p) % size

# Демонстрация различных методов хэширования
keys = [12345, 54321, 98765, 56789, 13579]
size = 100
p = 10007  # Простое число
a, b = 42, 17  # Случайные числа

print("Сравнение методов хэширования:")
print(f"{'Ключ':<10} {'Деление':<10} {'Умножение':<15} {'Универсальное':<15}")
print("-" * 50)

for key in keys:
    div_hash = division_hash(key, size)
    mul_hash = multiplication_hash(key, size)
    uni_hash = universal_hash(key, size, a, b, p)
    print(f"{key:<10} {div_hash:<10} {mul_hash:<15} {uni_hash:<15}")


- Необратимость (односторонность) - решение уравнения $h(x) = y$ относительно $x$ за $\Omega(|X|)$. На практике часто достаточно $\Omega(\sqrt{|X|})$

- Устойчивость к коллизиям - решение уравнения $h(x_1) = h(x_2)$ относительно $x_2$ за $\Omega(|X|)$

- Параметризуемость - целое семейство хеш-функций с конфигурируемыми параметрами : $\forall x, y \in Z_n$ $P(h_{\alpha}(x) = y) \le \frac{c}{n}$, где $c$ - константа и вероятность считается по случайному выбору параметров хеш функции и события $h_{\alpha}(w) = x$ независимы в совокупности.

- Специальные свойства
    - Модульность - хеш легко перевычисляется от частей объекта
    - Локальность - хеши схожих объектов слабо отличаются (противоположность лавинности)
    - etc

Если что, это просто список возможных свойств, не каждая хеш-функция должна обладать всеми

### 2. Криптографические хэш-функции

Криптографические хэш-функции должны удовлетворять более строгим требованиям безопасности:

1. **Стойкость к поиску прообраза**: Вычислительно невозможно найти сообщение $m$ по заданному хэш-значению $h(m)$.
2. **Стойкость к поиску второго прообраза**: Для заданного сообщения $m_1$ вычислительно невозможно найти другое сообщение $m_2 \neq m_1$ такое, что $h(m_1) = h(m_2)$.
3. **Стойкость к коллизиям**: Вычислительно невозможно найти любую пару различных сообщений $m_1 \neq m_2$ таких, что $h(m_1) = h(m_2)$.

#### Теорема о сложности атаки "грубой силой"

**Теорема:** Если хэш-функция $h$ выдает значения длиной $n$ бит и ведет себя как случайная функция, то:

1. Сложность нахождения прообраза составляет $O(2^n)$ операций.
2. Сложность нахождения коллизии составляет $O(2^{n/2})$ операций (согласно парадоксу дней рождения).

**Доказательство:**

Для нахождения прообраза необходимо перебрать в среднем половину всех возможных входных значений, что составляет $O(2^n)$ операций.

Для нахождения коллизии, согласно парадоксу дней рождения, достаточно вычислить хэш-значения для $O(\sqrt{2^n}) = O(2^{n/2})$ различных входных данных.


## Криптографические применения хэш-функций

Хэш-функции играют важную роль в криптографии и информационной безопасности. Рассмотрим некоторые из их основных применений.

### Цифровые подписи

Цифровая подпись — это криптографический механизм, который позволяет подтвердить подлинность и целостность сообщения или документа. Хэш-функции используются для создания "дайджеста" сообщения, который затем шифруется с помощью закрытого ключа отправителя.

**Теорема:** Если хэш-функция $h$ устойчива к коллизиям и алгоритм шифрования $E$ безопасен, то цифровая подпись $S(m) = E_{sk}(h(m))$ является защищенной от подделки, где $sk$ — закрытый ключ.

**Доказательство (схема):**

Предположим, что злоумышленник может создать действительную подпись для сообщения $m'$, которое он не подписывал. Это означает, что он может найти $S(m') = E_{sk}(h(m'))$ без знания $sk$. Возможны два случая:

1. Злоумышленник находит коллизию $h(m') = h(m)$ для некоторого известного подписанного сообщения $m$. Это противоречит предположению об устойчивости хэш-функции к коллизиям.

2. Злоумышленник может вычислить $E_{sk}(h(m'))$ без знания $sk$. Это противоречит предположению о безопасности алгоритма шифрования.

Таким образом, если оба предположения верны, цифровая подпись защищена от подделки.


In [ ]:
# Демонстрация цифровой подписи
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature

def generate_keys():
    """Генерирует пару ключей RSA"""
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(message, private_key):
    """Создает цифровую подпись для сообщения"""
    signature = private_key.sign(
        message.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

def verify_signature(message, signature, public_key):
    """Проверяет цифровую подпись"""
    try:
        public_key.verify(
            signature,
            message.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False

# Демонстрация
try:
    # Генерируем ключи
    private_key, public_key = generate_keys()

    # Сообщение для подписи
    message = "Это важное сообщение, которое требует подтверждения подлинности."

    # Создаем подпись
    signature = sign_message(message, private_key)

    print(f"Сообщение: {message}")
    print(f"Подпись (первые 20 байт): {signature[:20].hex()}")

    # Проверяем подпись
    is_valid = verify_signature(message, signature, public_key)
    print(f"Подпись действительна: {is_valid}")

    # Проверяем измененное сообщение
    modified_message = message + " Это дополнение было добавлено злоумышленником."
    is_valid = verify_signature(modified_message, signature, public_key)
    print(f"Подпись для измененного сообщения действительна: {is_valid}")

except ImportError:
    print("Библиотека cryptography не установлена. Установите ее с помощью 'pip install cryptography'")


### Хэширование паролей

Хэширование паролей — это метод защиты паролей пользователей в базе данных. Вместо хранения паролей в открытом виде, система хранит их хэш-значения. При аутентификации пользователя система хэширует введенный пароль и сравнивает результат с хранимым хэш-значением.

Для повышения безопасности используются следующие методы:

1. **Соль (salt)** — случайная строка, которая добавляется к паролю перед хэшированием. Это защищает от атак с использованием предварительно вычисленных таблиц (rainbow tables).

2. **Функции формирования ключа (KDF)** — специальные функции, которые намеренно требуют значительных вычислительных ресурсов, что затрудняет атаки методом перебора.

**Теорема:** Если хэш-функция $h$ устойчива к поиску прообраза и используется соль $s$, то вероятность успешной атаки методом перебора на хэш-значение $h(p \| s)$ для пароля $p$ пропорциональна $\frac{1}{|P| \cdot |S|}$, где $|P|$ — размер пространства паролей, а $|S|$ — размер пространства солей.

**Доказательство (схема):**

Без использования соли злоумышленник может предварительно вычислить хэш-значения для всех возможных паролей из пространства $P$. Это требует $O(|P|)$ вычислений и памяти.

С использованием соли для каждого пароля $p$ хранится уникальная соль $s$ и хэш-значение $h(p \| s)$. Злоумышленник должен перебирать пароли для каждой конкретной соли, что требует в среднем $O(|P|)$ вычислений для каждого пользователя, без возможности использования предварительных вычислений.


In [ ]:
# Демонстрация хэширования паролей
import hashlib
import os
import binascii
import time

def simple_hash_password(password):
    """Простое хэширование пароля без соли"""
    return hashlib.sha256(password.encode()).hexdigest()

def hash_password_with_salt(password):
    """Хэширование пароля с солью"""
    # Генерируем случайную соль
    salt = os.urandom(16)
    # Хэшируем пароль с солью
    hash_value = hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000)
    # Возвращаем соль и хэш в виде шестнадцатеричной строки
    return binascii.hexlify(salt).decode() + ':' + binascii.hexlify(hash_value).decode()

def verify_password(stored_hash, password):
    """Проверяет пароль по хэшу с солью"""
    # Разделяем соль и хэш
    salt, hash_value = stored_hash.split(':')
    # Преобразуем из шестнадцатеричной строки в байты
    salt = binascii.unhexlify(salt)
    hash_value = binascii.unhexlify(hash_value)
    # Вычисляем хэш для введенного пароля
    calculated_hash = hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000)
    # Сравниваем хэши
    return calculated_hash == hash_value

# Демонстрация
password = "секретный_пароль"

# Простое хэширование
simple_hash = simple_hash_password(password)
print(f"Простой хэш пароля: {simple_hash}")

# Хэширование с солью
start_time = time.time()
salted_hash = hash_password_with_salt(password)
end_time = time.time()
print(f"Хэш пароля с солью: {salted_hash}")
print(f"Время вычисления: {(end_time - start_time)*1000:.2f} мс")

# Проверка пароля
is_valid = verify_password(salted_hash, password)
print(f"Пароль верный: {is_valid}")

is_valid = verify_password(salted_hash, "неверный_пароль")
print(f"Неверный пароль принят: {is_valid}")


## Хэш-функции в распределенных системах

### Консистентное хэширование

Консистентное хэширование — это метод распределения данных между несколькими серверами, который минимизирует перераспределение данных при изменении количества серверов.

**Теорема:** В системе с консистентным хэшированием, использующей $k$ виртуальных узлов для каждого из $n$ серверов, при добавлении или удалении сервера перераспределяется в среднем $\frac{K}{n}$ ключей, где $K$ — общее количество ключей.

**Доказательство (схема):**

Пусть хэш-функция равномерно распределяет ключи по кольцу хэширования. Тогда каждый сервер отвечает за $\frac{1}{n}$ часть кольца и хранит в среднем $\frac{K}{n}$ ключей.

При добавлении нового сервера, он получает $\frac{1}{n+1}$ часть кольца, забирая примерно равные доли у существующих серверов. Таким образом, перераспределяется примерно $\frac{K}{n+1}$ ключей.

При удалении сервера, его $\frac{1}{n}$ часть кольца распределяется между оставшимися серверами. Таким образом, перераспределяется примерно $\frac{K}{n}$ ключей.


In [ ]:
# Реализация консистентного хэширования
import hashlib
import bisect

class ConsistentHashing:
    def __init__(self, nodes=None, replicas=100):
        """
        Инициализирует систему консистентного хэширования

        Параметры:
        nodes (list): Список узлов
        replicas (int): Количество виртуальных узлов для каждого реального узла
        """
        self.replicas = replicas
        self.ring = {}  # Хэш-значение -> узел
        self.sorted_keys = []  # Отсортированные хэш-значения

        if nodes:
            for node in nodes:
                self.add_node(node)

    def _hash(self, key):
        """Вычисляет хэш-значение ключа"""
        key_str = str(key).encode()
        return int(hashlib.md5(key_str).hexdigest(), 16)

    def add_node(self, node):
        """Добавляет узел в кольцо хэширования"""
        for i in range(self.replicas):
            key = self._hash(f"{node}:{i}")
            self.ring[key] = node
            bisect.insort(self.sorted_keys, key)

    def remove_node(self, node):
        """Удаляет узел из кольца хэширования"""
        for i in range(self.replicas):
            key = self._hash(f"{node}:{i}")
            if key in self.ring:
                del self.ring[key]
                self.sorted_keys.remove(key)

    def get_node(self, key):
        """Определяет узел для ключа"""
        if not self.ring:
            return None

        hash_key = self._hash(key)

        # Находим ближайший узел по часовой стрелке
        idx = bisect.bisect(self.sorted_keys, hash_key)
        if idx == len(self.sorted_keys):
            idx = 0

        return self.ring[self.sorted_keys[idx]]

    def get_distribution(self, keys):
        """Возвращает распределение ключей по узлам"""
        distribution = {}
        for key in keys:
            node = self.get_node(key)
            if node not in distribution:
                distribution[node] = 0
            distribution[node] += 1
        return distribution

# Демонстрация консистентного хэширования
# Создаем систему с 3 узлами
nodes = ["server1", "server2", "server3"]
ch = ConsistentHashing(nodes)

# Генерируем ключи
keys = [f"key{i}" for i in range(1000)]

# Получаем начальное распределение
initial_distribution = ch.get_distribution(keys)
print("Начальное распределение ключей:")
for node, count in initial_distribution.items():
    print(f"{node}: {count} ключей ({count/len(keys)*100:.1f}%)")

# Добавляем новый узел
ch.add_node("server4")

# Получаем новое распределение
new_distribution = ch.get_distribution(keys)
print("\nРаспределение после добавления узла:")
for node, count in new_distribution.items():
    print(f"{node}: {count} ключей ({count/len(keys)*100:.1f}%)")

# Подсчитываем количество перераспределенных ключей
moved_keys = 0
for key in keys:
    old_node = None
    for node, node_keys in initial_distribution.items():
        if key in [f"key{i}" for i in range(node_keys)]:
            old_node = node
            break

    new_node = ch.get_node(key)
    if old_node != new_node:
        moved_keys += 1

print(f"\nПерераспределено ключей: {moved_keys} ({moved_keys/len(keys)*100:.1f}%)")


## Заключение

Хэш-функции являются фундаментальным инструментом в информатике и криптографии, обеспечивая эффективное решение множества задач:

1. **Структуры данных**: Хэш-таблицы, фильтры Блума и другие структуры данных используют хэш-функции для быстрого доступа к информации.

2. **Криптография**: Криптографические хэш-функции обеспечивают целостность данных, аутентификацию и другие механизмы безопасности.

3. **Распределенные системы**: Консистентное хэширование и другие методы используют хэш-функции для эффективного распределения данных.

Теоретические основы хэш-функций, такие как парадокс дней рождения и теоремы о сложности атак, позволяют оценивать их безопасность и эффективность в различных приложениях.

Важно выбирать подходящую хэш-функцию в зависимости от требований конкретной задачи:

- Для структур данных приоритетом является скорость вычисления и равномерное распределение.
- Для криптографических приложений приоритетом является безопасность и устойчивость к различным типам атак.

Современные исследования в области хэш-функций направлены на разработку новых алгоритмов с улучшенными свойствами, а также на анализ безопасности существующих алгоритмов в контексте развития квантовых вычислений и других новых технологий.


## Библиография

1. Кормен Т., Лейзерсон Ч., Ривест Р., Штайн К. Алгоритмы: построение и анализ. — М.: Вильямс, 2013.

2. Шнайер Б. Прикладная криптография. Протоколы, алгоритмы, исходные тексты на языке С. — М.: Триумф, 2002.

3. Karger D., Lehman E., Leighton T., Panigrahy R., Levine M., Lewin D. Consistent Hashing and Random Trees: Distributed Caching Protocols for Relieving Hot Spots on the World Wide Web // Proceedings of the 29th Annual ACM Symposium on Theory of Computing. — 1997. — P. 654-663.

4. Bloom B. H. Space/Time Trade-offs in Hash Coding with Allowable Errors // Communications of the ACM. — 1970. — Vol. 13, № 7. — P. 422-426.

5. Preneel B. Analysis and Design of Cryptographic Hash Functions. — Katholieke Universiteit Leuven, 1993.

6. Menezes A. J., van Oorschot P. C., Vanstone S. A. Handbook of Applied Cryptography. — CRC Press, 1996.

7. Damgård I. B. Collision Free Hash Functions and Public Key Signature Schemes // Advances in Cryptology — EUROCRYPT '87. — 1987. — P. 203-216.

8. Merkle R. C. A Certified Digital Signature // Advances in Cryptology — CRYPTO '89. — 1989. — P. 218-238.
